In [18]:
import numpy as np
import cv2
import glob
import json
import dlib
import yaml
from imutils import face_utils
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras_vggface.vggface import VGGFace
from sklearn.decomposition import PCA, IncrementalPCA
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

In [3]:
participants = glob.glob('dataset/*')
# Define global variables
NUM_OF_PARTICIPANTS = len(participants)

In [7]:
with open('uuid_lengths.json') as json_file:
    uuid_lengths = json.load(json_file)

uuids = []
sum = 0
for i in range(NUM_OF_PARTICIPANTS):
    uuid = participants[i].split('/')[1]
    uuids.append(uuid)
    sum += uuid_lengths[uuid]
DATASET_SIZE = sum

In [8]:
print(DATASET_SIZE)

2710


In [4]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [27]:
def get_camera_parameters(uuid):
    with open("dataset/" + uuid + "/data.yml", 'r') as stream:
        yml_data = yaml.safe_load(stream)
        height = yml_data['height']
        width  = yml_data['width']
    size = [height, width]
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    return camera_matrix, dist_coeffs

In [71]:
# Accuracy metrics for the whole dataset. These are computed
# by leaving every Subject out one time, calculating the accuracy for each
# one and then taking the mean value.
# accuracy_rf_total = 0
accuracy_svm_total = 0
# precision_rf_total = 0
# precision_svm_total = 0
# recall_rf_total = 0
# recall_svm_total = 0


# Array to keep track of subjects with low score
# low_score_subjects_rf = []
# low_score_subjects_svm = []

num_features = 6

dataset_len = 0
# Loop over each participant
for j in range(5):
    
    # Extract the UUID
    uuid_excluded = uuids[j]
    length_excluded = uuid_lengths[uuid_excluded]
    
    features_train = np.zeros((DATASET_SIZE  - length_excluded + 18, num_features))
    features_val = np.zeros((length_excluded, num_features))
    labels_train = np.zeros(DATASET_SIZE - length_excluded + 18)
    labels_val = np.zeros(length_excluded)
    
    start = 0
        
    for i in range(NUM_OF_PARTICIPANTS):
        uuid = uuids[i]
        length = uuid_lengths[uuid]
        
        image_points_subj = np.load('FAN_landmarks/image_points_' + uuid + '.npy')
        model_points_subj = np.load('FAN_landmarks/model_points_' + uuid + '.npy')
        labels            = np.load('FAN_landmarks/labels_' + uuid + '.npy')
        print(np.sum(labels))
        
        if image_points_subj.shape[0] != length:
#             print('Error')
#             print(image_points_subj.shape[0], length, j, i, labels.shape[0])
#             print(uuid)
            length = image_points_subj.shape[0]
            
        features = np.zeros((length, num_features))

        
        camera_matrix, dist_coeffs = get_camera_parameters(uuid)
        for k in range(length):
            image_points = image_points_subj[k, :]
            model_points = model_points_subj[k, :]
            camera_matrix, dist_coeffs = get_camera_parameters(uuid)
            (success, rotation_vector, translation_vector) = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )
            
            features[k, :] = np.reshape(np.concatenate((rotation_vector, translation_vector), axis=0), (6, ))
                        
        if uuid == uuid_excluded:
            features_val = features
            labels_val   = labels

        else:
            features_train[start : start + length] = features
            labels_train[start : start + length] = labels
            start += length
    classifier = svm.SVC(C=10, kernel='rbf', gamma='scale', probability=True)
    classifier.fit(features_train, labels_train)
    
    # Predict SVM with threshold at 0.3 instead of 0.5
    threshold = 0.3
    y_prob_svm = classifier.predict_proba(features_val)
    y_pred_svm = (y_prob_svm[:, 1] >= threshold).astype(int)
    
#     classifier = RandomForestClassifier(n_estimators=100, random_state=1)
#     classifier.fit(features_train, labels_train)
#     y_pred_rf = rf_classifier.predict(features_val)

    confusion_matrix_svm = metrics.confusion_matrix(labels_val, y_pred_svm)
    
    accuracy_svm_subject = metrics.accuracy_score(labels_val, y_pred_svm)
    accuracy_svm_total += accuracy_svm_subject*y_pred_svm.shape[0]
    dataset_len += y_pred_svm.shape[0]
    
    print('#{} TEST SVM: {}, TRAIN SVM: {}'.format(j, round(accuracy_svm_subject, 2),
                                      round(classifier.score(features_train, labels_train), 2)))
    print(confusion_matrix_svm)
#     plot_learning_curve(classifier,'Learning Curve',  features_train, labels_train, cv=5)
#     plt.show()
print(accuracy_svm_total/dataset_len)

2.0
3.0
4.0
2.0
2.0
2.0
2.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
1.0
3.0
1.0
3.0
2.0
1.0
2.0
2.0
3.0
2.0
2.0
1.0
3.0
3.0
1.0
2.0
3.0
2.0
1.0
2.0
1.0
1.0
3.0
2.0
3.0
4.0
2.0
2.0
1.0
2.0
3.0
#0 TEST SVM: 1.0, TRAIN SVM: 1.0
[[31  0]
 [ 0  2]]
2.0
3.0
4.0
2.0
2.0
2.0
2.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
1.0
3.0
1.0
3.0
2.0
1.0
2.0
2.0
3.0
2.0
2.0
1.0
3.0
3.0
1.0
2.0
3.0
2.0
1.0
2.0
1.0
1.0
3.0
2.0
3.0
4.0
2.0
2.0
1.0
2.0
3.0
#1 TEST SVM: 1.0, TRAIN SVM: 1.0
[[28  0]
 [ 0  3]]
2.0
3.0
4.0
2.0
2.0
2.0
2.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
1.0
3.0
1.0
3.0
2.0
1.0
2.0
2.0
3.0
2.0
2.0
1.0
3.0


KeyboardInterrupt: 